# TP4: Travail Final 
## VANBELLE Julien

## Imports

In [22]:
import os
import yake
import string
import pandas as pd
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.corpus import webtext
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('webtext')
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import Image
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
from tabulate import tabulate

txt_path = '/Users/julienvanbelle/Documents/GitHub/tac/data/txt'
data_path = '/Users/julienvanbelle/Documents/GitHub/tac/data'


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/julienvanbelle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     /Users/julienvanbelle/nltk_data...
[nltk_data]   Package webtext is already up-to-date!


## Data cleaning

In [ ]:
files = os.listdir(txt_path)
data_bxl = [f for f in files]
len(data_bxl)


In [ ]:
data_list = []
for txt in data_bxl:
    with open(os.path.join(txt_path, txt), 'r', encoding = "ISO-8859-1") as f:
        data_list.append(f.read())

len(data_list)


In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
if not os.path.exists(data_path):
    os.mkdir(data_path)
with open(os.path.join(data_path, f'_temp.txt'), 'w') as f:
    f.write(' '.join(data_list))
    print("temp file saved in",data_path)

In [ ]:
wt_words = webtext.words('{}/_temp.txt'.format(data_path))
data_analysis = nltk.FreqDist(wt_words)
 
filter_words = dict([(m, n) for m, n in data_analysis.items() if len(m) > 3])

##for key in sorted(filter_words):
    ##print("%s: %s" % (key, filter_words[key]))
 
data_analysis = nltk.FreqDist(filter_words)
data_analysis.plot(30, cumulative=False)

In [ ]:
addsw = []
for z in sorted(filter_words):
  if filter_words[z] > 25000:
   ##print("%s: %s" % (z, filter_words[z]))
   addsw.append(z)

print(addsw)

In [ ]:
# Stopwords
swLower = ' '.join(str(e).lower() for e in addsw)
sw = stopwords.words("french")
sw += addsw
sw += swLower
sw += "conseil communal", "conseil général", "conseil supérieur", "administration communale", "conseil provincial", "l'administration communale", "conseil", "echevin" , "messieurs", "bruxelles", "bourgmestre", "collège", "être"
sw = set(sw)
print(sw)

In [ ]:
def clean_text(folder=None):
    if folder is None:
        input_path = f".txt"
        output_path = f"_clean.txt"
    else:
        input_path = f"{folder}/_temp.txt"
        output_path = f"{folder}/_clean.txt"
    output = open(output_path, "w", encoding="ISO-8859-1")
    with open(input_path, encoding="ISO-8859-1") as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(folder=data_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(data_path, f'_clean.txt'), 'r', encoding="ISO-8859-1") as f:
    after = f.read()

after[:5000]

## Analyse de la thématique des travaux

### TF/IDF

In [13]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [17]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=1.0,
    min_df=0.5,
    lowercase=True)

In [18]:
cld_data = [open(f'{data_path}/_clean.txt',encoding="ISO-8859-1").read()]

print("cld_data OK")


cld_data OK


In [19]:
%time tfidf_vectors = vectorizer.fit_transform(cld_data)

CPU times: user 3min 5s, sys: 6.64 s, total: 3min 11s
Wall time: 3min 15s


In [20]:
# Détail de la matrice
tfidf_vectors

<1x379056 sparse matrix of type '<class 'numpy.float64'>'
	with 379056 stored elements in Compressed Sparse Row format>

In [25]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

prã           0.367100
van           0.311157
rue           0.289534
collã         0.206637
âªtre         0.155175
                ...   
jeuitfc       0.000001
jeuke         0.000001
jeulx         0.000001
jeumaibien    0.000001
kunziger      0.000001
Length: 379056, dtype: float64